In [1]:
#read the dataset
import pandas as pd
df = pd.read_csv("C:/Users/sriva/Downloads/bc_230215.csv")
print(df.head()) 

   EVENT_NO_TRIP  EVENT_NO_STOP            OPD_DATE  VEHICLE_ID  METERS  \
0      259173279      259173350  15FEB2023:00:00:00        4223  194493   
1      259173279      259173350  15FEB2023:00:00:00        4223  194525   
2      259173279      259173350  15FEB2023:00:00:00        4223  194562   
3      259173279      259173350  15FEB2023:00:00:00        4223  194598   
4      259173279      259173350  15FEB2023:00:00:00        4223  194628   

   ACT_TIME  GPS_LONGITUDE  GPS_LATITUDE  GPS_SATELLITES  GPS_HDOP  
0     71279    -122.679572     45.517122             8.0       1.6  
1     71284    -122.679408     45.517392             8.0       1.9  
2     71290    -122.679227     45.517700             8.0       2.2  
3     71295    -122.679048     45.517998             9.0       0.9  
4     71300    -122.678888     45.518255             9.0       1.3  


In [2]:
#filtering using drop
df.drop(['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'], axis=1)
df

,EVENT_NO_TRIP,EVENT_NO_STOP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE,GPS_SATELLITES,GPS_HDOP
0,259173279,259173350,15FEB2023:00:00:00,4223,194493,71279,-122.679572,45.517122,8.0,1.6
1,259173279,259173350,15FEB2023:00:00:00,4223,194525,71284,-122.679408,45.517392,8.0,1.9
2,259173279,259173350,15FEB2023:00:00:00,4223,194562,71290,-122.679227,45.517700,8.0,2.2
3,259173279,259173350,15FEB2023:00:00:00,4223,194598,71295,-122.679048,45.517998,9.0,0.9
4,259173279,259173350,15FEB2023:00:00:00,4223,194628,71300,-122.678888,45.518255,9.0,1.3
...,...,...,...,...,...,...,...,...,...,...
2221397,258480928,258480932,15FEB2023:00:00:00,4304,109998,69191,-122.686300,45.594515,12.0,1.4
2221398,258480928,258480932,15FEB2023:00:00:00,4304,110041,69197,-122.686768,45.594723,12.0,0.8
2221399,258480928,258480932,15FEB2023:00:00:00,4304,110079,69202,-122.686840,45.595065,12.0,0.8
2221400,258480928,258480932,15FEB2023:00:00:00,4304,110114,69207,-122.686522,45.595262,12.0,0.8


In [3]:
#Filtering using usecols
columns_to_include = [0, 2, 3, 4, 5, 6, 7 ]
df = pd.read_csv("C:/Users/sriva/Downloads/bc_230215.csv", usecols=columns_to_include)
df

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
0,259173279,15FEB2023:00:00:00,4223,194493,71279,-122.679572,45.517122
1,259173279,15FEB2023:00:00:00,4223,194525,71284,-122.679408,45.517392
2,259173279,15FEB2023:00:00:00,4223,194562,71290,-122.679227,45.517700
3,259173279,15FEB2023:00:00:00,4223,194598,71295,-122.679048,45.517998
4,259173279,15FEB2023:00:00:00,4223,194628,71300,-122.678888,45.518255
...,...,...,...,...,...,...,...
2221397,258480928,15FEB2023:00:00:00,4304,109998,69191,-122.686300,45.594515
2221398,258480928,15FEB2023:00:00:00,4304,110041,69197,-122.686768,45.594723
2221399,258480928,15FEB2023:00:00:00,4304,110079,69202,-122.686840,45.595065
2221400,258480928,15FEB2023:00:00:00,4304,110114,69207,-122.686522,45.595262


In [4]:
from datetime import datetime, timedelta
df['OPD_DATE'] = pd.to_datetime(df['OPD_DATE'], format='%d%b%Y:%H:%M:%S')

group = df.groupby('EVENT_NO_TRIP')
dfs = []

for id, data in group:
    data = data.sort_values(by='ACT_TIME')
    
    data['TIMESTAMP'] = data.apply(lambda row: row['OPD_DATE'] + timedelta(seconds=row['ACT_TIME']), axis=1)
    data['dMETERS'] = data['METERS'].diff()
    data['dTIMESTAMP'] = data['TIMESTAMP'].diff()
    
    #Calculate the speed
    data['SPEED'] = data.apply(lambda row: row['dMETERS'] / row['dTIMESTAMP'].total_seconds() if row['dTIMESTAMP'].total_seconds() != 0 else 0, axis=1)
    print(f'\nTrip Number:{id}')
    speed = data['SPEED'].describe()
    print(f"{data['SPEED'].describe()}")
    print("\nMinimum speed:", speed['min'])
    print("Maximum speed:", speed['max'])
    print("Average speed:", speed['mean'])
    dfs.append(data)


df3 = pd.concat(dfs)
#drop the columns
df3 = df3.drop(['OPD_DATE', 'ACT_TIME','dMETERS', 'dTIMESTAMP'], axis=1)
print(df3)


Trip Number:258128451
count    317.000000
mean      11.051115
std        6.068407
min        0.000000
25%        7.000000
50%       10.500000
75%       14.200000
max       25.000000
Name: SPEED, dtype: float64

Minimum speed: 0.0
Maximum speed: 25.0
Average speed: 11.05111536728256

Trip Number:258128475
count    531.000000
mean       9.791489
std        5.983233
min        0.000000
25%        5.000000
50%        9.800000
75%       14.200000
max       26.600000
Name: SPEED, dtype: float64

Minimum speed: 0.0
Maximum speed: 26.6
Average speed: 9.79148940312369

Trip Number:258128535
count    298.000000
mean      12.755862
std        5.416218
min        0.000000
25%        9.800000
50%       13.600000
75%       16.400000
max       24.600000
Name: SPEED, dtype: float64

Minimum speed: 0.0
Maximum speed: 24.6
Average speed: 12.755861896772132

Trip Number:258135567
count    255.000000
mean      17.134779
std        8.138218
min        0.000000
25%       12.500000
50%       17.400000
75%  

In [5]:
print(df3['SPEED'].describe())
max_speed = df3['SPEED'].max()
print(f"\nThe maximum speed is: {max_speed}")

count    2.215666e+06
mean     8.741623e+00
std      5.401900e+00
min      0.000000e+00
25%      4.600000e+00
50%      8.800000e+00
75%      1.240000e+01
max      2.003333e+02
Name: SPEED, dtype: float64

The maximum speed is: 200.33333333333334


In [6]:
max_speed = df3['SPEED'].mean()
print(f"The mean speed is: {max_speed}")

The mean speed is: 8.741622835241047


In [17]:
group3 = df3.groupby(['VEHICLE_ID', 'EVENT_NO_TRIP'])

maxspeed = 0
mvehicle_id = None
mtrip_id = None
lat = None
lon = None

for (vehicle_id, trip_id), vehicle_data in group3:
    if not vehicle_data.empty:
        speed = vehicle_data['SPEED'].max()
        if speed > maxspeed:
            maxspeed = speed
            mvehicle_id = vehicle_id
            mtrip_id = trip_id
            lat = vehicle_data['GPS_LATITUDE'].iloc[0]
            lon = vehicle_data['GPS_LONGITUDE'].iloc[0]
            time = vehicle_data['TIMESTAMP'].iloc[0] 

print(f"Maximum Speed: \nVehicle ID: {mvehicle_id} \nTrip Number: {mtrip_id} \nMaximum Speed: {maxspeed} \nGPS Latitude: {lat} \nGPS Longitude: {lon} \nTime: {time}")

Maximum Speed: 
Vehicle ID: 3244 
Trip Number: 258750515 
Maximum Speed: 200.33333333333334 
GPS Latitude: 45.51734 
GPS Longitude: -122.869802 
Time: 2023-02-15 18:02:15


In [14]:
group3 = df3.groupby(['VEHICLE_ID', 'EVENT_NO_TRIP'])

meanspeed = 0
mvehicle_id = None
mtrip_id = None
lat = None
lon = None
time = None

for (vehicle_id, trip_id), vehicle_data in group3:
    if not vehicle_data.empty:
        speed = vehicle_data['SPEED'].mean()
        if speed > meanspeed:
            meanspeed = speed
            mvehicle_id = vehicle_id
            mtrip_id = trip_id
            lat = vehicle_data['GPS_LATITUDE'].iloc[0]  
            lon = vehicle_data['GPS_LONGITUDE'].iloc[0]
            time = vehicle_data['TIMESTAMP'].iloc[0] 

print(f"Average Speed: \nVehicle ID: {mvehicle_id} \nTrip Number: {mtrip_id} \nMean speed: {meanspeed} \nGPS Latitude: {lat} \nGPS Longitude: {lon} \nTime: {time}")

Average Speed: 
Vehicle ID: 3419 
Trip Number: 258614729 
Mean speed: 22.135467836257313 
GPS Latitude: 45.539397 
GPS Longitude: -122.377048 
Time: 2023-02-15 18:53:52
